# Main

In [ ]:
# Importing libraries
import os
import numpy as np
import pandas as pd
import sisl as si
import phonopy as ph
import matplotlib.pyplot as plt
# ASE modules
from ase import Atoms
from ase.calculators.siesta import Siesta
from ase.units import Ry
from ase.visualize import view
from ase.io import read, write
from ase.dft.kpoints import bandpath
# Path module
from pathlib import Path
# Matplotlib ticker
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
# Custom modules
#from codes.bulkBS import calcBS, calcPDOS
from src.cleanfiles import cleanFiles
from src.utils import SiestaProject
from src.structure import Perovskite
from src.parameterconv import run_siesta, basis_opt, grid_conv
from src.structureoptimizer import perovskite, relax_ase, relax_siesta
from src.bandscalc import calculate_bands, plot_bands, plot_bands_SISL
from src.phononcalc import calculate_phonons, plot_dispersion
from src.plotsettings import PlotSettings
PlotSettings().set_global_style()

## Contents
This notebook contains the following sections:

0. [Pseudopotentials and basis functions](#pseudo)
1. [Basis set optimization](#basisset)
2. [Real-space grid optimization](#spacegrid)
3. [Structural optimization (bulk)](#structureopt)
4. [Band structure and DOS (bulk)](#bands)
5. [Phonon dispersion (bulk)](#phonons)

<a id="pseudo"></a>
## 0. Pseudopotentials and basis functions

The main input of SIESTA are the pseudpotential files (.psml or .psf).\
In DFT we do not explicitly treat:
- tightly bound core electrons
- the nuclear Coulomb singularity

Instead, we replace nucleus + core electrons with an effective/pseudo potential acting on valence electrons.

We do this becouse core electrons:
- are chemically inert
- do not participate in bonding
- oscillate very rapidly near the nucleus
- require huge basis sets if treated explicitly

Removing them:
- makes calculations much cheaper
- keeps valence physics intact

When generating a pseudopotential, it is a decision about (valence configuration):
- which electrons are treated as valence
- which are frozen into the core

Pseudopotentials have been obtained from: https://www.pseudo-dojo.org/

The valence has been choosen as follows (same as in GPAW):
- Ba: 5s² 5p⁶ 6s² (valence = 10)
- Sr: 4s² 4p⁶ 5s² (valence = 10)
- Ti: 3s² 3p⁶ 3d² 4s² (valence = 12)
- O: 2s² 2p⁴ (valence = 6)

In [ ]:
from src.structure import Perovskite

In [ ]:
BaTiO3 = Perovskite('BaTiO3', a=4.00)
BaTiO3

In [ ]:
run_siesta(BaTiO3, xcf='PBEsol', basis='DZP', EnergyShift=0.01, SplitNorm=0.15,
           MeshCutoff=200, kgrid=(5, 5, 5), dir='resultsold/test')

In [ ]:
cleanFiles('resultsold/test', ['.nc', '.json'])

In [ ]:
sile = si.io.siesta.stdoutSileSiesta('resultsold/test/BaTiO3.out')

In [ ]:
# Read time in format 'hh:mm:ss' and convert to seconds
with open('resultsold/test/BaTiO3.out', 'r') as f:
    lines = f.readlines()
    t0 = lines[14].strip().split()[-1]
    t1 = lines[-2].strip().split()[-1]

print(f"Time taken for the run: {dt:.2f} seconds")

In [ ]:
sile

In [ ]:
sile

In [ ]:
sile = si.get_sile('resultsold/test/BaTiO3.fdf')
H = sile.read_hamiltonian()
# Create a short-hand to handle the geometry
geom = H.geometry
print(H)

In [ ]:
def plot_atom(atom):
    no = len(atom) # number of orbitals
    nx = no // 4
    ny = no // nx
    if nx * ny < no:
        nx += 1
    fig, axs = plt.subplots(nx, ny, figsize=(20, 5*nx))
    fig.suptitle('Atom: {}'.format(atom.symbol), fontsize=14)
    def my_plot(i, orb):
        grid = orb.toGrid(atom=atom, R=1)
        # Also write to a cube file
        #grid.write('{}_{}.cube'.format(atom.symbol, orb.name()))
        c, r = divmod(i, ny)
        if nx == 1:
            ax = axs[r]
        else:
            ax = axs[c][r]
        ax.imshow(grid.grid[:, :, grid.shape[2] // 2])
        ax.set_title(r'${}$'.format(orb.name(True)))
        ax.set_xlabel(r'$x$ [Ang]')
        ax.set_ylabel(r'$y$ [Ang]')
    i = 0
    for orb in atom:
        my_plot(i, orb)
        i += 1
    if i < nx * ny:
        # This removes the empty plots
        for j in range(i, nx * ny):
            c, r = divmod(j, ny)
            if nx == 1:
                ax = axs[r]
            else:
                ax = axs[c][r]
            fig.delaxes(ax)
        plt.draw()
plot_atom(geom.atoms[0])
plot_atom(geom.atoms[1])
plot_atom(geom.atoms[2])

<a id="basisset"></a>
## 1. Basis set optimization

Although the default basis sets generated by SIESTA might be enough for some applications, we start by doing some optimization of the basis sets, which can help achieve better results with similar computational costs.

We have 2 main parameters which allow rough control of the basis set quality:
- EnergyShift: Controls cut-off radii of the first-zeta orbitals: the lower the energy shift, the larger the cut-off radii.
- SplitNorm: Controls matching radii of the multiple-zeta orbitals: the lower the value, the larger the matching radii.

In [ ]:
BaTiO3 = perovskite('BaTiO3')
view(BaTiO3)

In [ ]:
# Extract total Energy
stdsile = si.io.siesta.stdoutSileSiesta('results/bulk/basis/BaTiO3.out')
Energies = stdsile.read_energy()
Etot = Energies['total']
print(f'Total Energy: {Etot:.3f} eV')
# Extract Forces
FAsile = si.get_sile('results/bulk/basis/BaTiO3.FA')
Forces = FAsile.read_force()
max_force = max(np.linalg.norm(f) for f in Forces)
print(f'Maximum Force: {max_force} eV/Å')
# Extract lattice constants
xvsile = si.get_sile('results/bulk/basis/BaTiO3.XV')
lat = xvsile.read_geometry().lattice
a = lat.length[0]
b = lat.length[1]
c = lat.length[2]
print(f'Lattice Constants: a={a:.3f} Å, b={b:.3f} Å, c={c:.3f} Å')
# Extract bandgap at the Gamma point
eig = si.get_sile("results/bulk/basis/BaTiO3.EIG").read_data()
Ef = Energies['fermi']
eig -= Ef
eig = eig.flatten()
VBM = eig[eig <= 0].max()
CBM = eig[eig > 0].min()
bandgap = CBM - VBM
print(f'Bandgap at Gamma: {bandgap:.3f} eV')

In [ ]:
stdsile.read_energy()['total']

In [ ]:
si.get_sile(f'results/bulk/basis/BaTiO3.FA').read_force()

In [ ]:
si.io.siesta.structSileSiesta('results/bulk/basis/BaTiO3.STRUCT_OUT').read_geometry()

In [ ]:
si.io.siesta.xvSileSiesta('results/bulk/basis/BaTiO3.XV').read_geometry().lattice

In [ ]:
sile = si.io.siesta.stdoutSileSiesta('results/bulk/basis/BaTiO3.out')
sile.read_energy()

In [ ]:
# This will run the basisoptimization script
# Note: This will take a long time to run on local, use cluster
#%run scripts/basis.py

In [ ]:
df = pd.read_csv('results/bulk/BaTiO3/basis/basisopt.csv')
df.tail()

In [ ]:

import matplotlib.pyplot as plt

def plot_basis(df, xaxis='SplitNorm', yaxis='Enthalpy'):
    # Find the column that is not xaxis to group by
    cols = df.columns[0:2]
    group_col = [col for col in cols if col != xaxis][0]

    # Figure 1 - yaxis vs xaxis for different values of group_col
    fig = plt.figure(figsize=[6, 5])
    ax = fig.add_subplot(111)
    # Plotting
    for val, data in df.groupby(group_col):
        x = data[xaxis]
        y = data[yaxis]
        y -= min(df[yaxis])
        y = y*1000/5
        ax.plot(x, y, marker='o', label=f'{group_col}={round(val, 3)}')
    # Add x- and y-labels
    ax.set_xlabel(xaxis)
    ax.set_ylabel(r'$\Delta$ ' + yaxis + r' (meV/atom)')
    # Add legend
    ax.legend()
    # Apply custom plot settings to the axes
    PlotSettings().set_style_ax(ax, gridlines=True)
    # Show the plot using tight layout
    plt.tight_layout()
    plt.show()

def plot_contour(df, vals='Enthalpy'):
    # Figure 2 - Contour plot of yaxis as a function of xaxis and group_col
    cols = df.columns[0:2]
    Z = df.pivot(index=cols[0], columns=cols[1], values=vals)
    Z -= np.nanmin(Z.values)
    Z = Z*1000
    if vals == 'Enthalpy':
        Z = Z/5

    X, Y = np.meshgrid(Z.columns.values, Z.index.values)
    # Plotting the log-likelihood contour
    fig = plt.figure(figsize=[6, 5])
    ax = fig.add_subplot(111)
    plt.title('Contour plot')
    CS = ax.contour(X,Y,Z, 100, cmap='viridis')
    fig.colorbar(CS, ax=ax, label=r'$\Delta$ ' + vals + r' (meV/atom)')
    # Add x- and y-labels
    ax.set_xlabel(cols[1])
    ax.set_ylabel(cols[0])

    ax.set_xticks(X[0,:], np.round(X[0,:], 3))
    ax.set_yticks(Y[:,0], np.round(Y[:,0], 3))

    #ax.set_xlim(0.1, 0.3)
    # Show the plot using tight layout
    plt.tight_layout()
    plt.show()

In [ ]:
plot_basis(df, xaxis='SplitNorm', yaxis='Enthalpy')

In [ ]:
plot_contour(df, vals='Enthalpy')

In [ ]:
from ase.calculators.siesta.parameters import Species, PAOBasisBlock

In [ ]:
basis_set = PAOBasisBlock(
"""1
0  2 S 0.2
0.0 0.0""")

In [ ]:
basis_set

<a id="spacegrid"></a>
## 2. Real-space grid convergence

In SIESTA, the real-space grid underpins accuracy and efficiency. Choosing the right cutoff is crucial: too low, and you risk inaccurate results and spurious egg-box effects (variation of the total energy as atoms are displaced with respect to the integration grid); too high, and the calculation becomes unnecessarily expensive.

In this section, we perform mesh-cutoff convergence

In [ ]:
# This will run the grid convergence script
# Note: This will take a long time to run on local, use cluster
#%run scripts/grid.py

In [ ]:
df = pd.read_csv('resultsold/bulk/grid/gridopt.csv', index_col=0)
df = df[df["MeshCutoff"]>=200]
df.head()


In [ ]:
fig, ax = plt.subplots(figsize=(7, 5))

for k, data in df.groupby('KPoints'):
    x = data['MeshCutoff']
    y = data['Enthalpy']*1000/5
    y -= min(y)
    ax.plot(x, y, marker='o', label=f'KPoints = {k}x{k}x{k}')

ax.set_xlabel('Mesh Cutoff (Ry)')
ax.set_ylabel(r'$\Delta$ Enthalpy (meV/atom)')
#ax.set_title('Basis Set Enthalpy vs SplitNorm for Different Energy Shift Values')
ax.legend()
ax.grid(True, which='both', linestyle='--', linewidth=0.5)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(7, 5))

for k, data in df.groupby('MeshCutoff'):
    x = data['KPoints']
    y = data['MaxForce']*1000
    y -= min(y)
    ax.plot(x, y, marker='o', label=f'MeshCutoff = {k} Ry')

ax.set_xlabel('KPoints')
ax.set_ylabel(r'$\Delta$ Max Force (meV/Å)')
#ax.set_title('Basis Set Enthalpy vs SplitNorm for Different Energy Shift Values')
ax.legend()
ax.grid(True, which='both', linestyle='--', linewidth=0.5)
plt.show()

<a id="structureopt"></a>
## 3. Structural optimization (bulk)

First we run a relaxation script where we let ASE handle position and cell optimization:

In [ ]:
#relax_ase(BaTiO3, xcf='PBEsol', basis='DZP', cutoff=50, kmesh=[5, 5, 5], mode='pw')

In [ ]:
atoms = read('results/bulk/relax/BaTiO3_pw.xyz')

In [ ]:
atoms.get_cell()

In [ ]:
atoms.get_positions()

The following function runs a single Siesta calculation which includes relaxation:

In [ ]:
#relax_siesta(BaTiO3, xcf='PBEsol', basis='DZP', cutoff=200, shift=0.008, kmesh=[5, 5, 5])

In [ ]:
atoms = read('results/bulk/relaxsiesta/BaTiO3.xyz')
atoms.get_cell()

In [ ]:
atoms.get_positions()

<a id="bands"></a>
## 4. Band structure and DOS (bulk)

The band structure can be calculated using either SIESTA or SISL.
The easiest is to use SISL, which can extract the Hamiltonian and geometry from the generated .HSX file:

In [ ]:
# Read relaxed structures
#BaTiO3_rel = read('bulk/relax/BaTiO3.xyz')
# Calculate band structure and PDOS
# Here commented out to avoid long computation, use cluster
#calculate_bands(BaTiO3, xcf='PBEsol', basis='DZP', cutoff=1200, shift=0.008, kmesh=[10, 10, 10])

In [ ]:
plot_bands('BaTiO3')

In [ ]:
plot_bands_SISL('BaTiO3')

In [ ]:
plot_bands('SrTiO3')

In [ ]:
plot_bands_SISL('SrTiO3')

<a id="phonons"></a>
## 5. Phonon dispersion (bulk)

We now move on to calculate the phonon dispersion. This is done with Phonopy using SIESTA as the calculator.

In [ ]:
# Read relaxed structures
#BaTiO3 = read('bulk/relax/BaTiO3.xyz')
#SrTiO3 = read('bulk/relax/SrTiO3.xyz')
# Calculate phonon dispersion
# Here commented out to avoid long computation, use cluster
#calculate_phonons(BaTiO3, xcf='PBEsol', basis='DZP', cutoff=1200, shift=0.008, kmesh=[5, 5, 5])

In [ ]:
#phonon.write_animation()

In [ ]:
phonon = ph.load(f'results/bulk/phonons/BaTiO3_phonon.yaml')
plot_dispersion(phonon)

In [ ]:
phonon = ph.load(f'results/bulk/phonons/BaTiO3.yaml')
plot_dispersion(phonon)

In [ ]:
phonon = ph.load(f'results/bulk/phonons/BaTiO3_pw_PBE.yaml')
plot_dispersion(phonon)

In [ ]:
phonon = ph.load(f'results/bulk/phonons/SrTiO3_phonon.yaml')
plot_dispersion(phonon)

<a id="slabrelax"></a>
## 6. Slab relaxation

In [ ]:
from src.structure import Perovskite

In [ ]:
atoms = Perovskite('BaTiO3', a=4.00, N=2, bulk=True).atoms
view(atoms)

<a id="bandsslab"></a>
## 7. Band structure and phonons in slabs

## Tests

In [ ]:
project = SiestaProject(material='BaTiO3')
df = project.get_summary()
df.tail()

In [ ]:
def filter_df(df, reference_id, param):
    # First cast all NaN values to 1
    df = df.fillna('1')
    # Find the row corresponding to the reference ID
    reference = df[df['ID'] == reference_id].iloc[0]
    # Find the columns to match (all columns except 'ID' and the parameter of interest)
    cols_to_match = df.columns.difference(['ID', param, 'phonons'])
    mask = (df[cols_to_match] == reference[cols_to_match]).all(axis=1)
    return df[mask].sort_values(by=param)

rdf = filter_df(df, '0065', 'SplitNorm')
#rdf = rdf[rdf['ID'].isin(['0053', '0051','0050', '0057', '0065'])]
ids = np.array(rdf['ID'])
vals = np.array(rdf['SplitNorm'])
rdf

In [ ]:
from src.bandscalc import plot_bands2
from src.phononcalc import plot_dispersion2

In [ ]:
dir = os.path.join('results/bulk/','BaTiO3', '0053', 'bands')
sile_PDOS = si.get_sile(os.path.join(dir, f"BaTiO3.PDOS"))
geom, E, PDOS = sile_PDOS.read_data()
PDOS = np.squeeze(PDOS)
orbits = geom.atoms.orbitals
DOS = PDOS.sum(axis=0)

In [ ]:
idx = np.cumsum(np.r_[0, orbits[:-1]])
PDOS_atom = np.add.reduceat(PDOS, idx, axis=0)

In [ ]:
atom_colors = {'Ba': 'tab:blue', 'Sr': 'tab:purple',
               'Ti': 'tab:orange', 'O': 'tab:red'}

In [ ]:
for i in range(PDOS_atom.shape[0]):
    symbol = geom.atoms[i].symbol
    plt.plot(E, PDOS_atom[i], color=atom_colors[symbol], label=f'{symbol}')
plt.xlabel('Energy (eV)')
plt.ylabel('PDOS (states/eV)')
plt.xlim(-6, 6)
plt.title('Projected Density of States for BaTiO3')
plt.legend()
plt.show()

In [ ]:
plot_bands('BaTiO3', ids=ids, vals=vals)
plot_bands2('BaTiO3', ids=ids, vals=vals)

In [ ]:
atoms = read('results/bulk/BaTiO3/0055/relax/BaTiO3.xyz')
view(atoms)

In [ ]:
rdf = filter_df(df, '0004', 'EnergyShift')
ids = np.array(rdf['ID'])
vals = np.array(rdf['EnergyShift'])
rdf

In [ ]:
plot_bands('SrTiO3')

In [ ]:
plot_bands('BaTiO3', ids=ids, vals=vals)

In [ ]:
plot_dispersion('BaTiO3', ids=ids, vals=vals, pDOS=False)

In [ ]:
from src.phononcalc import get_phonon_dispersion
def get_lattice(formula, calc_id):
    atoms = read(f'results/bulk/{formula}/{calc_id}/relax/{formula}.xyz')
    cell = atoms.get_cell()
    return cell

def get_phonon_frequencies(formula, calc_id):
    phonon = ph.load(f'results/bulk/{formula}/{calc_id}/phonons/{formula}.yaml')
    # Extract phonon dispersion data
    (dist, X, freq, labels) = get_phonon_dispersion(phonon, bulk=True)
    n_segments = len(freq)
    n_modes = freq[0].shape[1]

    fmode = [np.min(freq[0][0, :])]

    for i in range(n_segments):
        fmode.append(np.min(freq[i][-1, :]))
    # Return dictionary with labels and frequencies
    return dict(zip(labels, fmode))

In [ ]:
def plot_lattice(formula, reference_id, param):
    project = SiestaProject(material=formula)
    df = project.get_summary()
    rdf = filter_df(df, reference_id, param)
    ids = np.array(rdf['ID'])
    vals = np.array(rdf[param])
    lats = []
    for calc_id in ids:
        cell = get_lattice('BaTiO3', calc_id)
        a = cell[0, 0]
        b = cell[1, 1]
        c = cell[2, 2]
        lats.append((a, b, c))
    a_values = [lat[0] for lat in lats]
    b_values = [lat[1] for lat in lats]
    c_values = [lat[2] for lat in lats]

    fig = plt.figure()
    ax = fig.add_subplot(111)
    # Plot lattice constants vs parameter values
    ax.plot(vals, a_values, marker='o', label='a (Å)')
    ax.plot(vals, b_values, marker='o', label='b (Å)')
    ax.plot(vals, c_values, marker='o', label='c (Å)')
    # Add legend
    ax.legend()
    # Add x- and y-labels
    ax.set_xlabel(param)
    ax.set_ylabel('Lattice Constant (Å)')
    # Apply custom plot settings to the axes
    PlotSettings().set_style_ax(ax, gridlines=True)
    # Show the plot using tight layout
    plt.tight_layout()
    plt.show()

In [ ]:
def plot_phononfreq(formula, reference_id, param):
    project = SiestaProject(material=formula)
    df = project.get_summary()
    rdf = filter_df(df, reference_id, param)
    ids = np.array(rdf['ID'])
    vals = np.array(rdf[param])
    freq = []
    for calc_id in ids:
        frequens = get_phonon_frequencies(formula, calc_id)
        G = frequens['$\\Gamma$']
        X = frequens['X']
        R = frequens['R']
        M = frequens['M']
        freq.append((G, X, R, M))

    Gvals = [f[0] for f in freq]
    Xvals = [f[1] for f in freq]
    Rvals = [f[2] for f in freq]
    Mvals = [f[3] for f in freq]

    fig = plt.figure()
    ax = fig.add_subplot(111)
    # Plot phonon frequencies vs parameter values
    ax.plot(vals, Gvals, marker='o', label='$\\Gamma$')
    ax.plot(vals, Xvals, marker='o', label='X')
    ax.plot(vals, Rvals, marker='o', label='R')
    ax.plot(vals, Mvals, marker='o', label='M')
    # Add legend
    ax.legend()
    # Add x- and y-labels
    ax.set_xlabel(param)
    ax.set_ylabel('Phonon Frequency (THz)')
    # Apply custom plot settings to the axes
    PlotSettings().set_style_ax(ax, gridlines=True)
    # Show the plot using tight layout
    plt.tight_layout()
    plt.show()

In [ ]:
rdf = filter_df(df, '0041', 'pseudo')
ids = np.array(rdf['ID'])
vals = np.array(rdf['pseudo'])
rdf

In [ ]:
plot_lattice('BaTiO3', '0041', 'pseudo')
plot_phononfreq('BaTiO3', '0041', 'pseudo')